In [7]:
import pandas as pd

df = pd.read_csv('/content/tweetdata.csv',encoding='latin-1')


In [ ]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
#Splitting the main csv data file into train,val and test splits with 0.6,0.2, 0.2 respectively
from sklearn.model_selection import train_test_split

train, test_val = train_test_split(df, test_size=0.4, random_state=34)
val, test = train_test_split(test_val, test_size=0.5, random_state=34)

train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

In [9]:
train = pd.read_csv('train.csv',encoding='latin-1')
test = pd.read_csv('test.csv',encoding='latin-1')
val = pd.read_csv('val.csv',encoding='latin-1')

In [10]:
#Dropping the columns other than the classification label and content
train = train.drop(train.columns[[1, 2, 3, 4]], axis=1)
val = val.drop(val.columns[[1, 2, 3, 4]], axis=1)
test = test.drop(test.columns[[1, 2, 3, 4]], axis=1)

train.columns = ['Class', 'Tweet']
val.columns = ['Class', 'Tweet']
test.columns = ['Class', 'Tweet']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(train['Tweet'])

train_embeddings = tfidf_vectorizer.transform(train['Tweet'])
val_embeddings = tfidf_vectorizer.transform(val['Tweet'])
test_embeddings = tfidf_vectorizer.transform(test['Tweet'])



In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

train_labels = train['Class']
val_labels = val['Class']

clf = LogisticRegression()

param_grid = {'penalty': ['l2', 'none'],
              'C': [1.0,0.8],
              }

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=2, scoring='accuracy', verbose=10, n_jobs = -1)
grid_search.fit(train_embeddings, train_labels)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

val_predictions = grid_search.predict(val_embeddings)

print("Validation Classification Report:")
print(classification_report(val_labels, val_predictions))

Fitting 2 folds for each of 4 candidates, totalling 8 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best Parameters: {'C': 1.0, 'penalty': 'l2'}
Best Score: 0.7972497884804621
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80    159819
           4       0.79      0.81      0.80    160181

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000



In [13]:
from sklearn.metrics import classification_report

# Train SVM model with best hyperparameters
clf = LogisticRegression(**best_params, max_iter = 200)
clf.fit(train_embeddings, train_labels)

# Predictions on test data
test_predictions = clf.predict(test_embeddings)
test_labels = test['Class']

# Classification report
print("Test Classification Report:")
print(classification_report(test_labels, test_predictions))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80    160208
           4       0.80      0.81      0.80    159792

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X
  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [4]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

word2vec_file = '/content/glove.6B.100d.txt' + '.word2vec'
glove2word2vec('/content/glove.6B.100d.txt', word2vec_file)
model = KeyedVectors.load_word2vec_format(word2vec_file, binary=False)
vectorizer = Word2VecVectorizer(model)


<ipython-input-4-29ccfb26048d>:5: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec('/content/glove.6B.100d.txt', word2vec_file)


Loading in word vectors...
Finished loading in word vectors


In [14]:
import numpy as np
train_embeddings = vectorizer.fit_transform(train['Tweet'])
val_embeddings = vectorizer.fit_transform(val['Tweet'])
test_embeddings = vectorizer.fit_transform(test['Tweet'])

Numer of samples with no words found: 29045 / 959999
Numer of samples with no words found: 9728 / 320000
Numer of samples with no words found: 9532 / 320000


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

train_labels = train['Class']
val_labels = val['Class']

clf = LogisticRegression()

param_grid = {'penalty': ['l2', 'none'],
              'C': [1.0,0.8],
              }

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=2, scoring='accuracy', verbose=10, n_jobs = -1)
grid_search.fit(train_embeddings, train_labels)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

val_predictions = grid_search.predict(val_embeddings)

print("Validation Classification Report:")
print(classification_report(val_labels, val_predictions))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Best Parameters: {'C': 0.8, 'penalty': 'l2'}
Best Score: 0.6768507047801141
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.67      0.68    159819
           4       0.68      0.69      0.68    160181

    accuracy                           0.68    320000
   macro avg       0.68      0.68      0.68    320000
weighted avg       0.68      0.68      0.68    320000



In [16]:
from sklearn.metrics import classification_report

# Train SVM model with best hyperparameters
clf = LogisticRegression(**best_params, max_iter = 200)
clf.fit(train_embeddings, train_labels)

# Predictions on test data
test_predictions = clf.predict(test_embeddings)
test_labels = test['Class']

# Classification report
print("Test Classification Report:")
print(classification_report(test_labels, test_predictions))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.67      0.68    160208
           4       0.68      0.69      0.68    159792

    accuracy                           0.68    320000
   macro avg       0.68      0.68      0.68    320000
weighted avg       0.68      0.68      0.68    320000

